In [ ]:
  # Ver situação da GPU alocada
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-cadba3f3-d107-a3f3-894a-579d3e4cc55d)


In [ ]:
# Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Configuração dos paths
# path = ' /screws'
# path = '/content/drive/MyDrive/JKU/objectDetection/bikeParts'
path = '/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair'
# dataset_path = '/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets'
dataset_path = '/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair'

yolo_path = path + '/yolov5'
print (yolo_path)

/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5


# Área PERIGOSA

In [ ]:
# SOMENTE UMA VEZ

# Clonar yolov5
%cd '{path}'
!git clone https://github.com/ultralytics/yolov5

# !git clone --branch fix-exceeds-2gb https://github.com/zldrobit/yolov5
# !git clone https://github.com/LongxingTan/Yolov5.git

/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair
fatal: destination path 'yolov5' already exists and is not an empty directory.


# Instalação do YOLOv5

In [ ]:
# Instalar dependências do yolov5
%cd '{yolo_path}'
!pip install -U -r requirements.txt

/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install albumentations==1.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# %cd '{yolo_path}'
# import torch
# import utils
# display = utils.notebook_init()  # checks
import torch
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.12.1+cu113 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


# Treinamento

In [ ]:
# Executar tensorboard nos dados de treinamento (optional)
%load_ext tensorboard
%tensorboard --logdir '{yolo_path}/runs/train'

In [ ]:
# Definição dos parâmetros do teste
model_weights = 'yolov5l6'
model_batch = 10
# images_resolution = 1024
images_resolution = 640
epochs = 300
patience = 100
conf_threshold = 0.25

train_name = f'{model_weights}-{epochs}epochs-batch{model_batch}-resolution{images_resolution}-manual_augmentation'
print(train_name)

yolov5l6-300epochs-batch10-resolution640-manual_augmentation


In [ ]:
# Rodar treinamento
%cd '{yolo_path}'

# !python train.py --name '{train_name}' --resume --cache
# !python train.py --img '{images_resolution}' --batch '{model_batch}' --epochs '{epochs}' --weights '{model_weights}.pt' --data '../datasets/train.yaml' --name '{train_name}' --patience '{patience}'  --cache --exist-ok
!python train.py --img '{images_resolution}' --batch '{model_batch}' --epochs '{epochs}' --weights '{model_weights}.pt' --data '../train.yaml' --name '{train_name}' --patience '{patience}'  --cache --exist-ok


/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5
train: weights=yolov5l6.pt, cfg=, data=../train.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5l6-300epochs-batch10-resolution640-manual_augmentation, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera': No such file or directory
YOLOv5 🚀 2022-10-27 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB

# Inferência

In [ ]:
# Inferência no conjunto de testes
%cd '{yolo_path}'
!python detect.py --line-thickness 2 --weights 'runs/train/{train_name}/weights/best.pt' --img {images_resolution} --source ../datasets/test/images --conf {conf_threshold} --name '{train_name}_conf{conf_threshold}'  --save-txt --save-conf

/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5
detect: weights=['runs/train/yolov5l6-300epochs-batch10-resolution640-manual_augmentation/weights/best.pt'], source=../datasets/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=yolov5l6-300epochs-batch10-resolution640-manual_augmentation_conf0.25, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to '/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera': No such file or directory
YOLOv5 🚀 2022-10-27 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 346 layers, 76172508 parameters, 0 gradients, 110.0 GFLOPs
Traceback (most

# Métricas

In [ ]:
# Métricas no conjunto de testes
%cd '{yolo_path}'
!python val.py --weights 'runs/train/{train_name}/weights/best.pt' --img '{images_resolution}' --name '{train_name}' --data ../datasets/test.yaml

/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5
Traceback (most recent call last):
  File "val.py", line 405, in <module>
    opt = parse_opt()
  File "val.py", line 364, in parse_opt
    opt.data = check_yaml(opt.data)  # check YAML
  File "/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5/utils/general.py", line 422, in check_yaml
    return check_file(file, suffix)
  File "/content/drive/MyDrive/JKU/objectDetection/bikeParts/datasets/camera on the chair/yolov5/utils/general.py", line 448, in check_file
    assert len(files), f'File not found: {file}'  # assert file was found
AssertionError: File not found: ../datasets/test.yaml


# Exportar

In [ ]:
# Exportar
%cd '{yolo_path}'
!python export.py --weights 'runs/train/{train_name}/weights/best.pt' --include saved_model --img '{images_resolution}' --nms

# Enviar email no fim da execução

In [ ]:
import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "michelscheeren@alunos.utfpr.edu.br"
receiver_email = "michelhanzenscheeren@gmail.com"
password = "pgunxqslliojypti"
message = f"""From: From <michelscheeren@alunos.utfpr.edu.br>
To: To <michelhanzenscheeren@gmail.com>
Subject: Colab end

Your code execution was finished!
Name: {train_name}

"""

try:
  context = ssl.create_default_context()
  with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, message)
except Exception as error:
  print('Something went wrong!')
  print(error)

# Disconnect environment

In [ ]:
!pkill --oldest